In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

import os
import time
import datetime

from tensorflow.contrib import learn
from tensorflow.contrib.learn import preprocessing

/home/kierstenhenderson/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df = pd.read_csv('./data/dataset1_shortjokes_uci_cnn.csv')

In [3]:
#for lowercase 
df.text = df.text.apply(lambda x: x.lower())

In [4]:
df[:5]

,text,funny,not_funny
0,"[me narrating a documentary about narrators] ""...",1,0
1,telling my daughter garlic is good for you. go...,1,0
2,i've been going through a really rough period ...,1,0
3,"if i could have dinner with anyone, dead or al...",1,0
4,two guys walk into a bar. the third guy ducks.,1,0


In [5]:
np.random.seed(6)
shuffle = np.random.permutation(np.arange(df.shape[0]))

X = df['text']
Y_1 = df['funny']
Y_2 = df['not_funny'] 
X, Y_1, Y_2 = X[shuffle], Y_1[shuffle],Y_2[shuffle]
Y = pd.DataFrame({"funny": Y_1.values,"not_funny": Y_2.values})
Y.head()

,funny,not_funny
0,1,0
1,1,0
2,1,0
3,1,0
4,0,1


In [6]:
print('data shape: ', X.shape)
print('label shape:', Y.shape)


test_data, test_label = X[round(X.shape[0]*.9):], Y[round(Y.shape[0]*.9):]
dev_data, dev_label = X[round(X.shape[0]*.7):round(X.shape[0]*.9)], Y[round(Y.shape[0]*.7):round(Y.shape[0]*.9)]
train_data, train_label = X[:round(X.shape[0]*.7)], Y[:round(Y.shape[0]*.7)]


print('test_data', test_data.shape)
print('dev_data', dev_data.shape)
print('train_data', train_data.shape)

data shape:  (463314,)
label shape: (463314, 2)
test_data (46331,)
dev_data (92663,)
train_data (324320,)


In [9]:
#download and upzip Glove embeddings
if not os.path.exists('glove.42B.300d.zip'):
    ! wget http://nlp.stanford.edu/data/glove.42B.300d.zip
    ! unzip glove.42B.300d.zip
 
 

--2018-12-02 03:31:21--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2018-12-02 03:31:22--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Saving to: ‘glove.42B.300d.zip’

glove.42B.300d.zip  100%[===================>]   1.75G  6.75MB/s    in 2m 41s  

2018-12-02 03:34:03 (11.1 MB/s) - ‘glove.42B.300d.zip’ saved [1877800501/1877800501]

Archive:  glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [7]:
filename = 'glove.6B.300d.txt'

def loadGloVe(filename):
    vocab = []
    embd = []
    file = open(filename,'r')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append(row[1:])
    print('Loaded GloVe!')
    file.close()
    return vocab,embd
vocab,embd = loadGloVe(filename)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)

print(vocab_size)
print(embedding.shape)

Loaded GloVe!
400000
(400000, 300)


In [8]:
V = embedding.shape[0]

In [9]:
# Setup vocabulary processor to process embeddings and fit to train, dev, and test data

max_document_length = 34

#vocab_processor = tflearn.data_utils.VocabularyProcessor(max_length, min_frequency=0)


vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
#fit the vocab from glove
pretrain = vocab_processor.fit(vocab)
#transform inputs
#x = np.array(list(vocab_processor.transform(your_raw_input)))
x_train = np.array(list(vocab_processor.transform(train_data)))
x_dev = np.array(list(vocab_processor.transform(dev_data)))
x_test = np.array(list(vocab_processor.transform(test_data)))


Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.


In [10]:
y_train = np.array(train_label).astype(int)
y_dev = np.array(dev_label).astype(int)
y_test = np.array(test_label).astype(int)

print(y_train.shape)

(324320, 2)


In [11]:
print('corresponding ids\n',x_train[0])
print('corresponding ids\n',y_train[0])

print('corresponding ids\n',x_train[1])
print('corresponding ids\n',x_train[2])
print('corresponding ids\n',x_train[2000])
train_data.iloc[0]
train_data.iloc[0]

corresponding ids
 [   1 1586 3878    5    1   78   19 7102  130    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]
corresponding ids
 [1 0]
corresponding ids
 [ 126 3977    2 1727  711 1585 6630  169  809    5  809  223   10    5
  809  303  809    5   10  398   10    5   10    0    0    0    0    0
    0    0    0    0    0    0]
corresponding ids
 [12885    12   576  8868     5     0  2630     1  4679     5   184  1283
     4  1233  7394   184 95952    12 11751    25   576  5974     0     0
     0     0     0     0     0     0     0     0     0     0]
corresponding ids
 [  289     4     6  7649  2359    69     6  2044 50448 59064     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0]


'the worst journey in the world by helen back'

In [13]:
class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """

    def __init__(
      self, sequence_length, num_classes, vocab_size,
      embedding_size, pretrained_embedding, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
        
        
        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            
            #toggle true and false to train or not train Glove embeddings
            
            #from preassigned embedding W approach
            self.W=tf.get_variable(name="embedding_",shape=pretrained_embedding.shape,initializer=tf.constant_initializer(pretrained_embedding),trainable=False)
            
            self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                Wname = "w_%d"%filter_size
                #W = tf.get_variable(Wname, shape = filter_shape, initializer = tf.contrib.layers.xavier_initializer())
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                
                b = tf.Variable(tf.constant(0.0, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        
        #the line below differs from the tutorial, but is necessary based on changes to Tensorflow
        self.h_pool = tf.concat(pooled_outputs, 3)

        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.0, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            print("scores shape:", self.scores.shape)
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            
            #correct_predictions = tf.equal(tf.cast(self.predictions,tf.float32), self.input_y)
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))

            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")
            
    
        with tf.name_scope('train'):
            self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)

In [14]:
def batch_generator(ids, labels, batch_size=100):
            
    n_batches = len(ids)//batch_size
    ids, labels = ids[:n_batches*batch_size], labels[:n_batches*batch_size]
    shuffle = np.random.permutation(np.arange(n_batches*batch_size))
    ids, labels = ids[shuffle], labels[shuffle]

    
    for ii in range(0, len(ids), batch_size):
        yield ids[ii:ii+batch_size], labels[ii:ii+batch_size]
                
        

## Using Pretrained GloVe Embeddings, Switch to trainable = False

In [15]:
#Actual training loop:
embed_dim = 300 
filter_sizes= [3,4,5]
num_filters = 256
l2_reg_lambda = 0
learning_rate = 0.01
keep_prob = 0.5
evaluate_train = 1 # of epochs at which to print test accuracy
evaluate_dev = 1 # of epochs at which to estimate and print dev accuracy
time_print = 12 # of epochs at which to print time taken
num_classes = 2
num_epochs = 10

num_checkpoints = 1
batch_size = 64

out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", "cnn"))
print("Writing to {}\n".format(out_dir))

with tf.Graph().as_default():

    with tf.Session() as sess:
        
        cnn = TextCNN(sequence_length=x_train.shape[1], num_classes=num_classes, vocab_size=V, embedding_size=embed_dim, pretrained_embedding=embedding, filter_sizes= filter_sizes, 
                      num_filters=num_filters, l2_reg_lambda=l2_reg_lambda)
        
        
        sess.run(tf.global_variables_initializer())
        
        print('completed cnn creation')

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)

        

        print('# batches =', len(x_train)//batch_size)
        start = time.time()
        for e in range(num_epochs):
                    
            #sum_scores = np.zeros((batch_size*(len(x_train)//batch_size),1))
            total_loss = 0
            total_acc = 0
            for i, (x, y) in enumerate(batch_generator(x_train, y_train, batch_size), 1):
                feed = {cnn.input_x: x, cnn.input_y: y, cnn.dropout_keep_prob: keep_prob}
                _, scores, loss, accuracy = sess.run([cnn.optimizer,cnn.scores,cnn.loss, cnn.accuracy],feed_dict = feed)
                total_loss += loss*len(x)
                total_acc += accuracy*len(x)
                
                #sum_scores[i*batch_size:(i+1)*batch_size,:] = scores
                #print(np.mean(sum_scores))
                #time_str = datetime.datetime.now().isoformat()
            if e%evaluate_train==0:
                avg_loss = total_loss/(batch_size*(len(x_train)//batch_size))
                avg_acc = total_acc/(batch_size*(len(x_train)//batch_size))
                print("Train epoch {}, average loss {:g}, average accuracy {:g},".format(e, avg_loss, avg_acc))
                #print('average',np.mean(scores))

            if e%evaluate_dev==0:
                total_loss = 0
                total_acc = 0
                num_batches = 0
                for ii, (x, y) in enumerate(batch_generator(x_dev, y_dev, batch_size), 1):
                    feed_dict = {cnn.input_x: x, cnn.input_y: y, cnn.dropout_keep_prob: 1.0}
                    loss, accuracy = sess.run([cnn.loss, cnn.accuracy],feed_dict)
                    total_loss += loss*len(x)
                    total_acc += accuracy*len(x)
                    num_batches += 1
                avg_loss = total_loss/(num_batches*batch_size)
                avg_acc = total_acc/(num_batches*batch_size)
                #time_str = datetime.datetime.now().isoformat()
                print("\t\tDev epoch {}, average loss {:g}, average accuracy {:g},".format(e, avg_loss, avg_acc))
            if e%time_print == 0:
                end = time.time()
                print("\t\t\t\t    Time taken for",e,"epochs = ", end-start)

Writing to /home/kierstenhenderson/Final_project/runs/cnn

scores shape: (?, 2)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

completed cnn creation
# batches = 5067
Train epoch 0, average loss 1.03662, average accuracy 0.871164,
		Dev epoch 0, average loss 1.19198, average accuracy 0.89921,
				    Time taken for 0 epochs =  247.29960703849792
Train epoch 1, average loss 1.17292, average accuracy 0.887551,
		Dev epoch 1, average loss 0.668832, average accuracy 0.92167,
Train epoch 2, average loss 1.09064, average accuracy 0.894991,
		Dev epoch 2, average loss 1.18023, average accuracy 0.915558,
Train epoch 3, average loss 1.07578, average accuracy 0.900373,
		Dev epoch 3, average loss 1.17951, average accuracy 0.919413,
Train epoch 4, average loss 1.01848, average accuracy 0.902741,
		Dev epoch 4, average loss 0.867275, average accuracy 0.92

KeyboardInterrupt: 

## Using Pretrained GloVe Embeddings, Switch to trainable = True

In [15]:
class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(
      self, sequence_length, num_classes, vocab_size,
      embedding_size, pretrained_embedding, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
        
        
        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            
            #toggle true and false to train or not train Glove embeddings
            
            self.W=tf.get_variable(name="embedding_",shape=pretrained_embedding.shape,initializer=tf.constant_initializer(pretrained_embedding),trainable=True)
            
            
            self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                Wname = "w_%d"%filter_size
                #W = tf.get_variable(Wname, shape = filter_shape, initializer = tf.contrib.layers.xavier_initializer())
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                
                b = tf.Variable(tf.constant(0.0, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        
        #the line below differs from the tutorial, but is necessary based on changes to Tensorflow
        self.h_pool = tf.concat(pooled_outputs, 3)

        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.0, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            print("scores shape:", self.scores.shape)
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            
            #correct_predictions = tf.equal(tf.cast(self.predictions,tf.float32), self.input_y)
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))

            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")
          
        with tf.name_scope('train'):
            self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)
        

In [ ]:
#Actual training loop:

embed_dim = 300 
filter_sizes= [3,4,5]
num_filters = 256
l2_reg_lambda = 0
learning_rate = 0.01
keep_prob = 0.5
evaluate_train = 1 # of epochs at which to print test accuracy
evaluate_dev = 1 # of epochs at which to estimate and print dev accuracy
time_print = 12 # of epochs at which to print time taken
num_classes = 2
num_epochs = 10

num_checkpoints = 1
batch_size = 64

out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", "cnn"))
print("Writing to {}\n".format(out_dir))

with tf.Graph().as_default():

    with tf.Session() as sess:
        
        cnn = TextCNN(sequence_length=x_train.shape[1], num_classes=num_classes, vocab_size=V, embedding_size=embed_dim, pretrained_embedding=embedding, filter_sizes= filter_sizes, 
                      num_filters=num_filters, l2_reg_lambda=l2_reg_lambda)
        
        
        sess.run(tf.global_variables_initializer())
        
        
        print('completed cnn creation')

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)

        

        print('# batches =', len(x_train)//batch_size)
        start = time.time()
        for e in range(num_epochs):
                    
            #sum_scores = np.zeros((batch_size*(len(x_train)//batch_size),1))
            total_loss = 0
            total_acc = 0
            for i, (x, y) in enumerate(batch_generator(x_train, y_train, batch_size), 1):
                feed = {cnn.input_x: x, cnn.input_y: y, cnn.dropout_keep_prob: keep_prob}
                _, scores, loss, accuracy = sess.run([cnn.optimizer,cnn.scores,cnn.loss, cnn.accuracy],feed_dict = feed)
                total_loss += loss*len(x)
                total_acc += accuracy*len(x)
                
                #sum_scores[i*batch_size:(i+1)*batch_size,:] = scores
                #print(np.mean(sum_scores))
                #time_str = datetime.datetime.now().isoformat()
            if e%evaluate_train==0:
                avg_loss = total_loss/(batch_size*(len(x_train)//batch_size))
                avg_acc = total_acc/(batch_size*(len(x_train)//batch_size))
                print("Train epoch {}, average loss {:g}, average accuracy {:g},".format(e, avg_loss, avg_acc))
                #print('average',np.mean(scores))

            if e%evaluate_dev==0:
                total_loss = 0
                total_acc = 0
                num_batches = 0
                for ii, (x, y) in enumerate(batch_generator(x_dev, y_dev, batch_size), 1):
                    feed_dict = {cnn.input_x: x, cnn.input_y: y, cnn.dropout_keep_prob: 1.0}
                    loss, accuracy = sess.run([cnn.loss, cnn.accuracy],feed_dict)
                    total_loss += loss*len(x)
                    total_acc += accuracy*len(x)
                    num_batches += 1
                avg_loss = total_loss/(num_batches*batch_size)
                avg_acc = total_acc/(num_batches*batch_size)
                #time_str = datetime.datetime.now().isoformat()
                print("\t\tDev epoch {}, average loss {:g}, average accuracy {:g},".format(e, avg_loss, avg_acc))
            if e%time_print == 0:
                end = time.time()
                print("\t\t\t\t    Time taken for",e,"epochs = ", end-start)

Writing to /home/kierstenhenderson/Final_project/runs/cnn

scores shape: (?, 2)
completed cnn creation
# batches = 5067
Train epoch 0, average loss 0.781189, average accuracy 0.924854,
		Dev epoch 0, average loss 0.764221, average accuracy 0.944465,
				    Time taken for 0 epochs =  1812.7710220813751
Train epoch 1, average loss 0.745564, average accuracy 0.929387,
		Dev epoch 1, average loss 1.42296, average accuracy 0.946916,
Train epoch 2, average loss 0.802375, average accuracy 0.925125,
		Dev epoch 2, average loss 1.35228, average accuracy 0.945113,
Train epoch 3, average loss 0.845683, average accuracy 0.923855,
		Dev epoch 3, average loss 1.74177, average accuracy 0.936679,


## Using Pretrained GloVe Embeddings, Switch to trainable = True, change Learning Rate from 0.01 to 0.001



In [16]:
#Actual training loop:

embed_dim = 300 
filter_sizes= [3,4,5]
num_filters = 256
l2_reg_lambda = 0
learning_rate = 0.0001
keep_prob = 0.5
evaluate_train = 1 # of epochs at which to print test accuracy
evaluate_dev = 1 # of epochs at which to estimate and print dev accuracy
time_print = 12 # of epochs at which to print time taken
num_classes = 2
#num_epochs = 61
num_epochs = 10

num_checkpoints = 1
batch_size = 64

out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", "cnn"))
print("Writing to {}\n".format(out_dir))

with tf.Graph().as_default():

    with tf.Session() as sess:
        
        cnn = TextCNN(sequence_length=x_train.shape[1], num_classes=num_classes, vocab_size=V, embedding_size=embed_dim, pretrained_embedding=embedding, filter_sizes= filter_sizes, 
                      num_filters=num_filters, l2_reg_lambda=l2_reg_lambda)
        
        
        sess.run(tf.global_variables_initializer())
        
        
        print('completed cnn creation')

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)

       

        print('# batches =', len(x_train)//batch_size)
        start = time.time()
        for e in range(num_epochs):
                    
            #sum_scores = np.zeros((batch_size*(len(x_train)//batch_size),1))
            total_loss = 0
            total_acc = 0
            for i, (x, y) in enumerate(batch_generator(x_train, y_train, batch_size), 1):
                feed = {cnn.input_x: x, cnn.input_y: y, cnn.dropout_keep_prob: keep_prob}
                _, scores, loss, accuracy = sess.run([cnn.optimizer,cnn.scores,cnn.loss, cnn.accuracy],feed_dict = feed)
                total_loss += loss*len(x)
                total_acc += accuracy*len(x)
                
                #sum_scores[i*batch_size:(i+1)*batch_size,:] = scores
                #print(np.mean(sum_scores))
                #time_str = datetime.datetime.now().isoformat()
            if e%evaluate_train==0:
                avg_loss = total_loss/(batch_size*(len(x_train)//batch_size))
                avg_acc = total_acc/(batch_size*(len(x_train)//batch_size))
                print("Train epoch {}, average loss {:g}, average accuracy {:g},".format(e, avg_loss, avg_acc))
                #print('average',np.mean(scores))

            if e%evaluate_dev==0:
                total_loss = 0
                total_acc = 0
                num_batches = 0
                for ii, (x, y) in enumerate(batch_generator(x_dev, y_dev, batch_size), 1):
                    feed_dict = {cnn.input_x: x, cnn.input_y: y, cnn.dropout_keep_prob: 1.0}
                    loss, accuracy = sess.run([cnn.loss, cnn.accuracy],feed_dict)
                    total_loss += loss*len(x)
                    total_acc += accuracy*len(x)
                    num_batches += 1
                avg_loss = total_loss/(num_batches*batch_size)
                avg_acc = total_acc/(num_batches*batch_size)
                #time_str = datetime.datetime.now().isoformat()
                print("\t\tDev epoch {}, average loss {:g}, average accuracy {:g},".format(e, avg_loss, avg_acc))
            if e%time_print == 0:
                end = time.time()
                print("\t\t\t\t    Time taken for",e,"epochs = ", end-start)

Writing to /home/kierstenhenderson/Final_project/runs/cnn

scores shape: (?, 2)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

completed cnn creation
# batches = 5067
Train epoch 0, average loss 0.310542, average accuracy 0.886101,
		Dev epoch 0, average loss 0.118627, average accuracy 0.956645,
				    Time taken for 0 epochs =  1899.1284637451172
Train epoch 1, average loss 0.111261, average accuracy 0.958568,
		Dev epoch 1, average loss 0.0815744, average accuracy 0.969625,
Train epoch 2, average loss 0.0715139, average accuracy 0.973289,
		Dev epoch 2, average loss 0.0673643, average accuracy 0.97497,
Train epoch 3, average loss 0.0514264, average accuracy 0.981233,
		Dev epoch 3, average loss 0.0619873, average accuracy 0.977518,
Train epoch 4, average loss 0.0385358, average accuracy 0.98609,
		Dev epoch 4, average loss 0.0608853, averag